# Prática 06 - Bibliotecas de visualização geográficas

In [43]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import json
import time
import os
import re
import fileinput
from folium.plugins import Search
import altair as alt



In [57]:
ceara = gpd.read_file("data/geojs-23-mun.json")
popCeara = pd.read_csv("data/ceara.csv")

ceara['centroid_lon'] = ceara['geometry'].centroid.x
ceara['centroid_lat'] = ceara['geometry'].centroid.y

centerLon = np.mean(ceara['centroid_lon'])
centerLat = np.mean(ceara['centroid_lat'])
numCidades = len(ceara)
print(numCidades)
ceara['populacao'] = popCeara["POPULAÇÃO"]

184


In [111]:
ceara

,id,name,description,geometry,centroid_lon,centroid_lat,populacao
0,2300101,Abaiara,Abaiara,"POLYGON ((-38.96262 -7.40021, -38.96994 -7.425...",-39.037166,-7.359844,10496
1,2300150,Acarape,Acarape,"POLYGON ((-38.65581 -4.15804, -38.62384 -4.165...",-38.662995,-4.221969,15338
2,2300200,Acaraú,Acaraú,"POLYGON ((-40.31697 -2.80534, -40.26131 -2.810...",-40.088974,-2.967569,57551
3,2300309,Acopiara,Acopiara,"POLYGON ((-39.56759 -5.98316, -39.56142 -6.008...",-39.492500,-6.121981,51160
4,2300408,Aiuaba,Aiuaba,"POLYGON ((-40.10700 -6.35117, -40.08474 -6.368...",-40.298389,-6.604728,16203
...,...,...,...,...,...,...,...
179,2313807,Uruburetama,Uruburetama,"POLYGON ((-39.47198 -3.64954, -39.51421 -3.668...",-39.515521,-3.622423,19765
180,2313906,Uruoca,Uruoca,"POLYGON ((-40.69208 -3.24210, -40.66917 -3.247...",-40.679827,-3.320883,12883
181,2313955,Varjota,Varjota,"POLYGON ((-40.44576 -4.16348, -40.44220 -4.183...",-40.494383,-4.167643,17593
182,2314003,Várzea Alegre,Várzea Alegre,"POLYGON ((-39.21471 -6.59585, -39.20952 -6.607...",-39.297919,-6.771896,38434


In [94]:
ceara2 = pd.DataFrame()

In [116]:
ceara2 = ceara.loc[ceara['name'].isin(["Ararendá", "Catunda", "Crateús", 
                                       "Hidrolândia", "Independência", "Ipaporanga", 
                                       "Ipueiras", "Monsenhor Tabosa", "Nova Russas",
                                       "Novo Oriente", "Poranga", "Santa Quitéria", "Tamboril"])]
ceara2

,id,name,description,geometry,centroid_lon,centroid_lat,populacao
14,2301257,Ararendá,Ararendá,"POLYGON ((-40.65669 -4.70846, -40.64571 -4.720...",-40.746452,-4.772595,10491
42,2303659,Catunda,Catunda,"POLYGON ((-40.04537 -4.45580, -40.03672 -4.456...",-40.174787,-4.631361,9952
49,2304103,Crateús,Crateús,"POLYGON ((-40.59437 -4.95025, -40.56661 -5.018...",-40.719364,-5.198539,72812
69,2305209,Hidrolândia,Hidrolândia,"POLYGON ((-40.24756 -4.47991, -40.24557 -4.487...",-40.402233,-4.438882,19325
77,2305605,Independência,Independência,"POLYGON ((-40.18667 -5.16783, -40.18825 -5.175...",-40.341514,-5.474096,25573
78,2305654,Ipaporanga,Ipaporanga,"POLYGON ((-40.84225 -4.78553, -40.83276 -4.803...",-40.803509,-4.909622,11343
81,2305902,Ipueiras,Ipueiras,"POLYGON ((-40.77416 -4.43657, -40.73344 -4.464...",-40.876944,-4.572914,37862
115,2308609,Monsenhor Tabosa,Monsenhor Tabosa,"POLYGON ((-39.92454 -4.73488, -39.92822 -4.774...",-40.044173,-4.914783,16705
122,2309300,Nova Russas,Nova Russas,"POLYGON ((-40.37439 -4.55353, -40.42273 -4.661...",-40.528040,-4.679364,30965
123,2309409,Novo Oriente,Novo Oriente,"POLYGON ((-40.58418 -5.46703, -40.59696 -5.490...",-40.770946,-5.592460,27453


In [117]:

alt.Chart(ceara2).mark_geoshape().encode( alt.Color('populacao', type='quantitative')
).properties(
    width=500,
    height=300
)


alt.Chart(...)

In [ ]:
fp = "data/estados.json"
dadosHomicidios = "data/homicidios_2019.csv"
#fp = "data/municipios_simp.json"
gdf = gpd.read_file(fp)
homicidios = pd.read_csv(dadosHomicidios,delimiter=',')
#gdf = gpd.GeoDataFrame.from_features(customGeoJson)
homicidios

Aqui fazemos a conversão do objeto `GeoJson` em um [Pandas](www.pandas.pydata.org) `DataFrame`. Posteriormente, determinamos os centroides de cada estado e o centro do mapa

In [ ]:
gdf['HOMICIDIOS']=gdf['GEOCODIGO'].astype(int)
gdf['centroid_lon'] = gdf['geometry'].centroid.x
gdf['centroid_lat'] = gdf['geometry'].centroid.y
centerLon = np.mean(gdf['centroid_lon'])
centerLat = np.mean(gdf['centroid_lat'])
numEstados = len(gdf)
numEstados

Para checar a geometria de cada estado

In [ ]:
gdf['geometry'][6]

In [ ]:
gdf['HOMICIDIOS'] = homicidios['NumHom']

Usamos os dados `centerLon` e `centerLat` para definir as coordenadas centrais do mapa:

In [ ]:
mapa = folium.Map(location=[centerLat,centerLon],zoom_start=5,control_scale=True)

In [ ]:
g1 = folium.Choropleth(
    geo_data=fp,
    name='choropleth',
    data=gdf,
    columns=['GEOCODIGO', 'HOMICIDIOS'],
    key_on='feature.properties.GEOCODIGO',
    fill_color='RdGy',
    fill_opacity=0.7,
    line_opacity=0,
    bins=6,
    smooth_factor=2,
    legend_name='HOMICIDIOS EM 2019',
    highlight=True
).add_to(mapa)

`GeoJson` para dar **Highlight** nos objetos selecionados

In [ ]:
g1 = folium.GeoJson(
        gdf[['NOME', 'geometry','GEOCODIGO']].to_json(), 
        name='Teste Mapa',
        show=True,
        style_function=lambda feature: {
            'fillColor': 'green',
            'color': 'white',
            'weight': 0,
            'fillOpacity':0
        },   
        highlight_function=lambda x: {'weight':0.1, 
                                      'color':'black',
                                      'fillOpacity':0.2},    
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NOME','GEOCODIGO'],
            aliases=['Cidade:','Código'],                              
        ),  
).add_to(mapa)

In [ ]:
citysearch = Search(
    layer=g1,
    geom_type='Point',
    placeholder='Busca Cidade',
    collapsed=True,
    search_label='NOME'
).add_to(mapa)

In [ ]:
mapa.save("mapaT.html") 

In [ ]:
mapa